# Setup
(No need to change anything)

In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-i0vpqi05
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-i0vpqi05
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 174209ea708fe3838ccf08b70f2f4f28e7397cb4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
    

In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [3]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [4]:
# import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

We turn automatic differentiation off, to save GPU memory, as this notebook focuses on model inference not model training.

In [5]:
torch.set_grad_enabled(False)

Plotting helper functions:

In [6]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

## Load Model

Decide which model to use (eg. gpt2-small vs -medium)

In [7]:
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

Loaded pretrained model gpt2-small into HookedTransformer


## Import functions from repo

In [8]:
!git clone https://github.com/callummcdougall/ARENA_2.0.git

Cloning into 'ARENA_2.0'...
remote: Enumerating objects: 9100, done.
remote: Counting objects: 100% (1814/1814), done.
remote: Compressing objects: 100% (288/288), done.
remote: Total 9100 (delta 1609), reused 1602 (delta 1523), pack-reused 7286
Receiving objects: 100% (9100/9100), 155.60 MiB | 23.78 MiB/s, done.
Resolving deltas: 100% (5502/5502), done.
Updating files: 100% (365/365), done.


In [9]:
cd ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification

/content/ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification


In [10]:
import ioi_circuit_extraction as ioi_circuit_extraction

# Generate dataset with multiple prompts

In [19]:
class Dataset:
    def __init__(self, prompts, pos_dict, tokenizer, S1_is_first=False):
        self.prompts = prompts
        self.tokenizer = tokenizer
        self.N = len(prompts)
        self.max_len = max(
            [
                len(self.tokenizer(prompt["text"]).input_ids)
                for prompt in self.prompts
            ]
        )
        # all_ids = [prompt["TEMPLATE_IDX"] for prompt in self.ioi_prompts]
        all_ids = [0 for prompt in self.prompts] # only 1 template
        all_ids_ar = np.array(all_ids)
        self.groups = []
        for id in list(set(all_ids)):
            self.groups.append(np.where(all_ids_ar == id)[0])

        texts = [ prompt["text"] for prompt in self.prompts ]
        self.toks = torch.Tensor(self.tokenizer(texts, padding=True).input_ids).type(
            torch.int
        )
        self.io_tokenIDs = [
            self.tokenizer.encode(" " + prompt["corr"])[0] for prompt in self.prompts
        ]
        self.s_tokenIDs = [
            self.tokenizer.encode(" " + prompt["incorr"])[0] for prompt in self.prompts
        ]

        # word_idx: for every prompt, find the token index of each target token and "end"
        # word_idx is a tensor with an element for each prompt. The element is the targ token's ind at that prompt
        self.word_idx = {}
        for targ in [key for key in self.prompts[0].keys() if (key != 'text' and key != 'corr' and key != 'incorr')]:
            targ_lst = []
            for prompt in self.prompts:
                input_text = prompt["text"]
                tokens = model.tokenizer.tokenize(input_text)
                # if S1_is_first and targ == "S1":  # only use this if first token doesn't have space Ġ in front
                #     target_token = prompt[targ]
                # else:
                #     target_token = "Ġ" + prompt[targ]
                # target_index = tokens.index(target_token)
                target_index = pos_dict[targ]
                targ_lst.append(target_index)
            self.word_idx[targ] = torch.tensor(targ_lst)

        targ_lst = []
        for prompt in self.prompts:
            input_text = prompt["text"]
            tokens = self.tokenizer.tokenize(input_text)
            end_token_index = len(tokens) - 1
            targ_lst.append(end_token_index)
        self.word_idx["end"] = torch.tensor(targ_lst)

    def __len__(self):
        return self.N

Repalce io_tokens with correct answer (next, which is '5') and s_tokens with incorrect (current, which repeats)

In [22]:
import string

def generate_prompts_list(x, y):
    prompts_list = []
    letters = list(string.ascii_uppercase)[:y - x]
    for i, letter in enumerate(letters, start=x):
        if i == len(letters)-1:
            continue

        prompt_dict = {
            'S1': letter,
            'S2': letter,
            'S3': letter,
            'S4': letter,
            'S5': letter,
            'S6': letter,
            'corr': letter,
            'incorr': letters[i + 1],
            'text': f"{letter} {letter} {letter} {letter} {letter} {letter}"
        }
        pos_dict = {
            'S1': 0,
            'S2': 1,
            'S3': 2,
            'S4': 3,
            'S5': 4,
            'S6': 5,
        }
        prompts_list.append(prompt_dict)
    return prompts_list, pos_dict

prompts_list, pos_dict = generate_prompts_list(0, 11)
dataset = Dataset(prompts_list, pos_dict, model.tokenizer, S1_is_first=True)

In [50]:
def generate_prompts_list_corr(x, y):
    prompts_list = []
    letter = list(string.ascii_uppercase)
    for i in range(10):
        prompt_dict = {
            'S1': letter[i],
            'S2': letter[i+1],
            'S3': letter[i+2],
            'S4': letter[i+3],
            'S5': letter[i+4],
            'S6': letter[i+5],
            'corr': letter[i+6],
            'incorr': letter[i+5],
            'text': f"{letter[i]} {letter[i+1]} {letter[i+2]} {letter[i+3]} {letter[i+4]} {letter[i+5]}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

pos_dict = {
    'S1': 0,
    'S2': 1,
    'S3': 2,
    'S4': 3,
    'S5': 4,
    'S6': 5,
}
prompts_list_2 = generate_prompts_list_corr(0, 11)
dataset_2 = Dataset(prompts_list_2, pos_dict, model.tokenizer, S1_is_first=True)

In [52]:
len(prompts_list_2)

10

The clean and corrupted prompt lists (size of dataset) MUST be the same size for the activations to replace; this is due to batch size being the first dim of z (actv vector) and means vector. Batch size is init using the dataset size.

Logit diff is correct - incorr token. Here, correct is S5, and incorr is S4.

Because of this, it's possible to have logit diffs HIGHER than the "full circuit" because the correct token will still be at first place, but the logit scores assigned will just be bigger (perhaps incorrect is scored even lower in the non-full circuit with a higher logit diff score)?

# Ablation Expm Functions

In [31]:
from torch import Tensor

def logits_to_ave_logit_diff_2(logits: Float[Tensor, "batch seq d_vocab"], dataset: Dataset, per_prompt=False):
    '''
    Returns logit difference between the correct and incorrect answer.

    If per_prompt=True, return the array of differences rather than the average.
    '''

    # Only the final logits are relevant for the answer
    # Get the logits corresponding to the indirect object / subject tokens respectively
    io_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.io_tokenIDs]
    s_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.s_tokenIDs]
    # Find logit difference
    answer_logit_diff = io_logits - s_logits
    return answer_logit_diff if per_prompt else answer_logit_diff.mean()

In [32]:
def mean_ablate_by_lst(CIRCUIT, SEQ_POS_TO_KEEP, model, print_output=True):
    model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook

    ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)

    model = ioi_circuit_extraction.add_mean_ablation_hook(model, means_dataset=dataset_2, circuit=CIRCUIT, seq_pos_to_keep=SEQ_POS_TO_KEEP)
    ioi_logits_minimal = model(dataset.toks)

    orig_score = logits_to_ave_logit_diff_2(ioi_logits_original, dataset)
    new_score = logits_to_ave_logit_diff_2(ioi_logits_minimal, dataset)
    if print_output:
        # print(f"Average logit difference (IOI dataset, using entire model): {orig_score:.4f}")
        # print(f"Average logit difference (IOI dataset, only using circuit): {new_score:.4f}")
        print(f"Average logit difference (circuit / full) %: {100 * new_score / orig_score:.4f}")
    # return new_score
    return 100 * new_score / orig_score

# test prompts

In [27]:
modeltest = HookedTransformer.from_pretrained("gpt2-small")

Loaded pretrained model gpt2-small into HookedTransformer


In [28]:
example_prompt = "A A A A A A"
example_answer = " A"
utils.test_prompt(example_prompt, example_answer, modeltest, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'A', ' A', ' A', ' A', ' A', ' A']
Tokenized answer: [' A']


Performance on answer token:
Rank: 0        Logit: 14.65 Prob: 64.31% Token: | A|

Top 0th token. Logit: 14.65 Prob: 64.31% Token: | A|
Top 1th token. Logit: 12.41 Prob:  6.81% Token: |
|
Top 2th token. Logit: 12.08 Prob:  4.93% Token: | B|
Top 3th token. Logit: 10.18 Prob:  0.73% Token: | C|
Top 4th token. Logit:  9.96 Prob:  0.59% Token: | D|
Top 5th token. Logit:  9.88 Prob:  0.54% Token: |

|
Top 6th token. Logit:  9.46 Prob:  0.36% Token: | S|
Top 7th token. Logit:  9.45 Prob:  0.36% Token: | -|
Top 8th token. Logit:  9.29 Prob:  0.30% Token: |.|
Top 9th token. Logit:  9.19 Prob:  0.27% Token: | P|


Ranks of the answer tokens: [(' A', 0)]

In [48]:
example_prompt = "A B C D E F"
example_answer = " F"
utils.test_prompt(example_prompt, example_answer, modeltest, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'A', ' B', ' C', ' D', ' E', ' F']
Tokenized answer: [' F']


Performance on answer token:
Rank: 15       Logit: 12.81 Prob:  0.01% Token: | F|

Top 0th token. Logit: 22.55 Prob: 98.53% Token: | G|
Top 1th token. Logit: 17.26 Prob:  0.50% Token: | 1|
Top 2th token. Logit: 17.02 Prob:  0.39% Token: | g|
Top 3th token. Logit: 15.54 Prob:  0.09% Token: |g|
Top 4th token. Logit: 15.02 Prob:  0.05% Token: |G|
Top 5th token. Logit: 14.72 Prob:  0.04% Token: | H|
Top 6th token. Logit: 14.11 Prob:  0.02% Token: | +|
Top 7th token. Logit: 14.00 Prob:  0.02% Token: | GO|
Top 8th token. Logit: 13.65 Prob:  0.01% Token: | I|
Top 9th token. Logit: 13.49 Prob:  0.01% Token: |
|


Ranks of the answer tokens: [(' F', 15)]

We can also prevent redundant computation of the full circuit score by storing it and just passing it in to the function.

# Ablate the model and compare with original

In [51]:
def circuit_from_headsList(headsList):
    CIRCUIT = {
        "number mover": headsList,
        "number mover 6": headsList,
        "number mover 5": headsList,
        "number mover 4": headsList,
        "number mover 3": headsList,
        "number mover 2": headsList,
        "number mover 1": headsList,
    }

    SEQ_POS_TO_KEEP = {
        "number mover": "end",
        "number mover 6": "S6",
        "number mover 5": "S5",
        "number mover 4": "S4",
        "number mover 3": "S3",
        "number mover 2": "S2",
        "number mover 1": "S1",
    }
    return CIRCUIT, SEQ_POS_TO_KEEP

headsList = [(layer, head) for layer in range(12) for head in range(12)]
CIRCUIT, SEQ_POS_TO_KEEP = circuit_from_headsList(headsList)
mean_ablate_by_lst(CIRCUIT, SEQ_POS_TO_KEEP, model, print_output=False).item()

100.0

## Work backwards

In [53]:
# Start with full circuit
curr_circuit = [(layer, head) for layer in range(12) for head in range(12)]
threshold = 3  # This is T, a %. if performance is less than T%, allow its removal

for layer in range(11, -1, -1):  # go thru all heads in a layer first
    for head in range(12):
        # Copying the curr_circuit so we can iterate over one and modify the other
        copy_circuit = curr_circuit.copy()

        # Temporarily removing the current tuple from the copied circuit
        copy_circuit.remove((layer, head))

        CIRCUIT, SEQ_POS_TO_KEEP = circuit_from_headsList(copy_circuit)
        new_score = mean_ablate_by_lst(CIRCUIT, SEQ_POS_TO_KEEP, model, print_output=False).item()

        # print((layer,head), new_score)
        # If the result is less than the threshold, remove the tuple from the original list
        if (100 - new_score) < threshold:
            curr_circuit.remove((layer, head))

            print("Removed:", (layer, head))
            print(new_score)
            print("\n")

Removed: (11, 0)
102.46851348876953


Removed: (11, 1)
102.49149322509766


Removed: (11, 2)
101.9510269165039


Removed: (11, 3)
101.07994842529297


Removed: (11, 5)
101.44995880126953


Removed: (11, 6)
100.6679916381836


Removed: (11, 7)
100.29447174072266


Removed: (11, 8)
100.15706634521484


Removed: (11, 9)
100.94445037841797


Removed: (11, 10)
118.32706451416016


Removed: (11, 11)
116.91827392578125


Removed: (10, 0)
116.7237777709961


Removed: (10, 1)
119.06087493896484


Removed: (10, 2)
117.02185821533203


Removed: (10, 3)
116.32490539550781


Removed: (10, 4)
116.4767837524414


Removed: (10, 5)
116.37277221679688


Removed: (10, 6)
116.52081298828125


Removed: (10, 7)
156.48748779296875


Removed: (10, 8)
154.5279083251953


Removed: (10, 9)
154.41763305664062


Removed: (10, 10)
105.39897918701172


Removed: (10, 11)
105.40689849853516


Removed: (9, 0)
104.7690200805664


Removed: (9, 1)
111.53325653076172


Removed: (9, 2)
111.37552642822266


Removed: (9, 3)
1

In [54]:
curr_circuit

[(0, 1),
 (0, 10),
 (1, 0),
 (1, 4),
 (1, 5),
 (1, 9),
 (2, 2),
 (2, 3),
 (2, 8),
 (2, 9),
 (2, 10),
 (3, 0),
 (3, 7),
 (3, 8),
 (3, 10),
 (3, 11),
 (4, 8),
 (4, 10),
 (4, 11),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 5),
 (5, 6),
 (6, 3),
 (6, 6),
 (6, 7),
 (6, 8),
 (6, 9),
 (6, 10),
 (7, 8),
 (7, 10),
 (7, 11),
 (8, 1),
 (8, 3),
 (8, 5),
 (8, 6),
 (8, 7),
 (8, 8),
 (8, 11),
 (9, 11),
 (11, 4)]

In [55]:
len(curr_circuit)

43

In [ ]:
repLet = [(0, 1), (0, 10), (1, 0), (1, 4), (1, 5), (1, 9), (2, 2), (2, 3), (2, 8), (2, 9), (2, 10), (3, 0), (3, 7), (3, 8), (3, 10), (3, 11), (4, 8), (4, 10), (4, 11), (5, 0), (5, 1), (5, 2), (5, 3), (5, 5), (5, 6), (6, 3), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (7, 8), (7, 10), (7, 11), (8, 1), (8, 3), (8, 5), (8, 6), (8, 7), (8, 8), (8, 11), (9, 11), (11, 4)]

In [ ]:
repDig = [(0, 1), (0, 2), (0, 3), (0, 5), (0, 7), (0, 9), (1, 0), (1, 5), (1, 7), (2, 2), (2, 9), (3, 6), (3, 7), (3, 9), (5, 1), (9, 9), (10, 1), (10, 2), (10, 3), (10, 5), (10, 6), (11, 4), (11, 8)]

In [ ]:
ioi_dict = {
    "name mover": [(9, 9), (10, 0), (9, 6)],
    "backup name mover": [(10, 10), (10, 6), (10, 2), (10, 1), (11, 2), (9, 7), (9, 0), (11, 9)],
    "negative name mover": [(10, 7), (11, 10)],
    "s2 inhibition": [(7, 3), (7, 9), (8, 6), (8, 10)],
    "induction": [(5, 5), (5, 8), (5, 9), (6, 9)],
    "duplicate token": [(0, 1), (0, 10), (3, 0)],
    "previous token": [(2, 2), (4, 11)]
}

IOIcirc = [item for sublist in ioi_dict.values() for item in sublist]
IOIcirc

In [ ]:
print(len(curr_circuit))
print(len(IOIcirc))

23
26


In [ ]:
rep_not_IOI = list(set(curr_circuit) - set(IOIcirc))
print(sorted(rep_not_IOI))
len(rep_not_IOI)

[(0, 2), (0, 3), (0, 5), (0, 7), (0, 9), (1, 0), (1, 5), (1, 7), (2, 9), (3, 6), (3, 7), (3, 9), (5, 1), (10, 3), (10, 5), (11, 4), (11, 8)]


17

In [ ]:
IOI_not_rep = list(set(IOIcirc) - set(curr_circuit))
print(sorted(IOI_not_rep))
len(IOI_not_rep)

[(0, 10), (3, 0), (4, 11), (5, 5), (5, 8), (5, 9), (6, 9), (7, 3), (7, 9), (8, 6), (8, 10), (9, 0), (9, 6), (9, 7), (10, 0), (10, 7), (10, 10), (11, 2), (11, 9), (11, 10)]


20